In [1]:
# importing required liabraries
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import csv
import nltk

In [0]:
#Inintializing ELMO embeddinf from Tensorflow Hub

elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [3]:
#Connecting to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Reading Dataset from Google Drive
df=pd.read_csv("/content/drive/My Drive/Precily/Text_Similarity_Dataset.csv")

In [5]:
#Showing top 5 tuple of the dataframe
df.head()

,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...


In [6]:
# Checking dataset for null values  

df.isnull().sum()

Unique_ID    0
text1        0
text2        0
dtype: int64

In [7]:
#Converting dataframe to list columwise 

txt1=df['text1'].values.tolist()
txt2=df['text2'].values.tolist()



print('Total Tuples in Dataframe is ',len(df))
print('Total Tuples in Text1 is ',len(txt1))
print('Total Tuples in Text2 is ',len(txt2))

Total Tuples in Dataframe is  4023
Total Tuples in Text1 is  4023
Total Tuples in Text2 is  4023


In [8]:
#Downloading required data for preprocessing

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.tokenize import word_tokenize # To tokenize sentence to words
from nltk.stem import WordNetLemmatizer # To Lemmatize word
from nltk.corpus import stopwords       # Common stopwords

In [0]:
stopwords=set(stopwords.words("english")) # Loading stopwords to variable
lemmatizer=WordNetLemmatizer() # Initializing lemmatizer

In [0]:
def stopword(sentence): # Funtion to remove stopwords from text.
    token_words=word_tokenize(sentence)
    token_words
    stop_sentence=[]
    for word in token_words:
      if word not in stopwords:
        stop_sentence.append(word)
        stop_sentence.append(" ")
    return "".join(stop_sentence)


def lemm(sentence): # Funtion to lemmatize text.
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(lemmatizer.lemmatize(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def preprocess(sentence): # Funtion to preprocess text.
    return lemm(stopword(sentence))



In [0]:
# Preprocess text and saving to other list

text1=[]
text2=[]

for i in range(len(df)):
  text1.append(preprocess(txt1[i]))
  text2.append(preprocess(txt2[i]))


In [14]:
print("Text before preprocessing")
print(txt1[0])

print("Text After preprocessing")
print(text1[0])

Text before preprocessing
savvy searchers fail to spot ads internet search engine users are an odd mix of naive and sophisticated  suggests a report into search habits.  the report by the us pew research center reveals that 87% of searchers usually find what they were looking for when using a search engine. it also shows that few can spot the difference between paid-for results and organic ones. the report reveals that 84% of net users say they regularly use google  ask jeeves  msn and yahoo when online.  almost 50% of those questioned said they would trust search engines much less  if they knew information about who paid for results was being hidden. according to figures gathered by the pew researchers the average users spends about 43 minutes per month carrying out 34 separate searches and looks at 1.9 webpages for each hunt. a significant chunk of net users  36%  carry out a search at least weekly and 29% of those asked only look every few weeks. for 44% of those questioned  the inf

# Using Elmo embedding

In [0]:
# Converting text to vector using ELMO on Tensorflow Hub

def elmo_vectors(x):
   
  embeddings=elmo(x, signature="default", as_dict=True)["elmo"]
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
import gc
from tqdm import tnrange
csv_filename = 'Semantic_Record'
elmo_embeddings_text1=[]
elmo_embeddings_text2=[]
simlarity=[]
simlarity_matrix=[]


for i in tnrange(len(df)):
  
    #Computing vector and calculating simlarity using cosin simlarity
    
    elmo_embeddings_text1.append(elmo_vectors([text1[i]])[0])
    elmo_embeddings_text2.append(elmo_vectors([text2[i]])[0])
    simlarity_matrix=cosine_similarity(elmo_embeddings_text1, elmo_embeddings_text2)
    simlarity.append(float (simlarity_matrix[0]))
    simlarity[0]=round((simlarity[0]-1)*(-1),4)
    
    with open(csv_filename,'a', newline='') as simlarity_file:
        writer = csv.writer(simlarity_file)
        writer.writerow(simlarity) #Saving simlarity of text in csv file
    del simlarity[:]
    del simlarity_matrix
    del elmo_embeddings_text1[:]
    del elmo_embeddings_text2[:]
    gc.collect()                  #Garbage collection to free memory 

In [17]:
# Reading Simlarity values computed above

Semantic_Record=pd.read_csv("Semantic_Record.csv")

# Top 5 values of dataset

Semantic_Record.head()

,Simlarity
0,0.4825
1,0.2848
2,0.2401
3,0.2284
4,0.2128


In [27]:
# Storing simlarity values in list

simlarity=Semantic_Record['Simlarity'].values.tolist()
print("Total number of simlarity count is",len(simlarity))


Total number of simlarity count is 4023


In [0]:
# Storing Unique_ID values in list

unique=df['Unique_ID'].values.tolist()

# Ziping both the list in a dataframe 

result=pd.DataFrame(list(zip(unique,simlarity)),columns =['Unique_ID','Similarity_Score'],index=None) 


In [0]:
# Storing result in a csv file
result.to_csv('Similarity_Scores.csv')